Nous allons créer un Decision Tree permettant de classifier differents elements, en l'occurence des fruits en fonctions de trois attributs.

In [3]:
from __future__ import print_function
import numpy as np
import pandas as pd

In [4]:
training_data = pd.read_csv("data.csv")

### Visualisez les donnees (valeurs reelles) ###

In [5]:
training_data

,Color,Size,Season,Label
0,Red,1,Summer,Grape
1,Green,3,Spring,Apple
2,Red,3,Winter,Orange
3,Green,2,Winter,Kiwi
4,Yellow,3,Winter,Lemon
5,Green,2,Winter,Kiwi
6,Yellow,3,Winter,Lemon
7,Green,1,Summer,Grape
8,Red,3,Spring,Apple
9,Red,3,Winter,Orange


## Definissez X qui contiendra les attributs ET les labels (pour faciliter le partinioning par la suite)##

In [6]:
X = training_data.values[:,:4]

In [7]:
# Nous devons maintenant stocker les differents titres de categrories pour pouvoir afficher notre arbre
header = ["color", "size", "season", "label"]

## Definir la fonction permettant de trouver les differentes valeurs possibles pour chaque categorie ##

In [8]:
def unique_vals(rows, col):
    return set([row[col] for row in rows])

### Testez votre fonction pour les differents attributs et labels ###

In [9]:
# Quelles sont les differentes saisons ?
unique_vals(X, 2)

{'Spring', 'Summer', 'Winter'}

In [10]:
# Quels sont les couleurs possibles ?
unique_vals(X, 0)

{'Green', 'Red', 'Yellow'}

In [11]:
# Quels sont les differents fruits presents dans le dataset ?
unique_vals(X, -1)

{'Apple', 'Grape', 'Kiwi', 'Lemon', 'Orange'}

### Definir la fonction permettant de compter le nombre de fruits présents *(astuce : créer un dictionnaire)* ###

In [12]:
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [13]:
## Lancez votre fonction qui prend en paramètre X :
class_counts(X)
## Elle devrait retourner : {'Grape': 19, 'Apple': 19, 'Orange': 20, 'Kiwi': 22, 'Lemon': 20} 

{'Grape': 19, 'Apple': 19, 'Orange': 20, 'Kiwi': 22, 'Lemon': 20}

### Definir la fonction is_numeric permettant de savoir si une valeur est de type int/float ou non. ###

In [14]:
def is_numeric(value):
    if type(value) == int or type(value) == float:
        return True
    else:
        return False

In [15]:
## Testez votre fonction pour une valeur numerique (int ou float) ...
is_numeric(7.09)

True

In [16]:
#... et pour une valeur non-numérique 
is_numeric("Red")

False

### Pour partitionner le dataset, nous allons utiliser unr classe "Questions" ! Cette classe enregistre un numero de colonne (Ex : 0 pour la couleur) et une valeur de la colonne (ex : Vert).  La fonction match est utilisee pour comparee la valeur d'un element a celle de la question. .###

In [42]:
class Question:
    
    def __init__(self, column, value):
        # Le Constructeur  permet d'attribuer les attributs à la classe
        self.column = column
        self.value = value

    def match(self, example):
        # Compare l'attribut dans un exemple à l'attribut dans la question
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

#### Testez vos questions

In [47]:
# Ecrivez une questions pour un attribut numerique
q1 = Question(1,2)
q1

Is size >= 2?

In [48]:
# Et une deuxieme pour un attribut catégorique : 
q2 = Question(2, "Winter")
q2

Is season == Winter?

In [62]:
# Choisissons un example dans le dataset ...
example = X[0]
# ... Et testez si votre exemple est bien de couleur rouge
q3 = Question(0, "Red")
q3.match(X[0])

True

### Grace a cette classe, nous allons pouvoir partitionner le dataset ###

In [64]:
def partition(rows, question):
    # Pour chaque ligne dans le dataset, partionnez le en 2 en fonction du resultat de la question.
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [66]:
# Testons notre fonction partition pour isoler les lignes "rouges"
true_rows, false_rows = partition(X, Question(0, 'Red'))
true_rows

[array(['Red', 1, 'Summer', 'Grape'], dtype=object),
 array(['Red', 3, 'Winter', 'Orange'], dtype=object),
 array(['Red', 3, 'Spring', 'Apple'], dtype=object),
 array(['Red', 3, 'Winter', 'Orange'], dtype=object),
 array(['Red', 1, 'Summer', 'Grape'], dtype=object),
 array(['Red', 3, 'Winter', 'Orange'], dtype=object),
 array(['Red', 1, 'Summer', 'Grape'], dtype=object),
 array(['Red', 3, 'Winter', 'Orange'], dtype=object),
 array(['Red', 1, 'Summer', 'Grape'], dtype=object),
 array(['Red', 3, 'Winter', 'Orange'], dtype=object),
 array(['Red', 1, 'Summer', 'Grape'], dtype=object),
 array(['Red', 3, 'Winter', 'Orange'], dtype=object),
 array(['Red', 1, 'Summer', 'Grape'], dtype=object),
 array(['Red', 3, 'Winter', 'Orange'], dtype=object),
 array(['Red', 1, 'Summer', 'Grape'], dtype=object),
 array(['Red', 3, 'Winter', 'Orange'], dtype=object),
 array(['Red', 1, 'Summer', 'Grape'], dtype=object),
 array(['Red', 3, 'Winter', 'Orange'], dtype=object),
 array(['Red', 1, 'Summer', 'Grape'],

In [67]:
#... et les autres
false_rows

[array(['Green', 3, 'Spring', 'Apple'], dtype=object),
 array(['Green', 2, 'Winter', 'Kiwi'], dtype=object),
 array(['Yellow', 3, 'Winter', 'Lemon'], dtype=object),
 array(['Green', 2, 'Winter', 'Kiwi'], dtype=object),
 array(['Yellow', 3, 'Winter', 'Lemon'], dtype=object),
 array(['Green', 1, 'Summer', 'Grape'], dtype=object),
 array(['Green', 3, 'Winter', 'Kiwi'], dtype=object),
 array(['Yellow', 2, 'Winter', 'Lemon'], dtype=object),
 array(['Green', 3, 'Spring', 'Apple'], dtype=object),
 array(['Green', 2, 'Winter', 'Kiwi'], dtype=object),
 array(['Yellow', 2, 'Winter', 'Lemon'], dtype=object),
 array(['Green', 3, 'Spring', 'Apple'], dtype=object),
 array(['Green', 3, 'Winter', 'Kiwi'], dtype=object),
 array(['Yellow', 3, 'Winter', 'Lemon'], dtype=object),
 array(['Green', 3, 'Spring', 'Apple'], dtype=object),
 array(['Green', 2, 'Winter', 'Kiwi'], dtype=object),
 array(['Yellow', 3, 'Winter', 'Lemon'], dtype=object),
 array(['Green', 3, 'Spring', 'Apple'], dtype=object),
 array(['G

#### Maintenant un peu de maths !
#### Nous allons calculer l'impurity de chaque categorie (true / false) grace a GINI

In [71]:
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [72]:
# Pour verifier notre comprehension, testez votre gini avec l'exemple suivant 
no_mixing = [['Apple'], ['Apple']]
# Que devrait-etre votre resultat ?
gini(no_mixing)

0.0

In [73]:
 # Maintenant avec un exemple comportant deux valeurs
some_mixing = [['Apple'], ['Orange'], ['Orange'], ['Apple']]
#  Que devriez vous obtenir comme resultat ?
gini(some_mixing)

0.5

In [74]:
# Et maintenant pour un dataset avec tous nos fruits
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Lemon'],
                  ['Kiwi']]
# Que devriez vous obtenir comme resultat ?
gini(lots_of_mixing)

0.7999999999999998

### Calculons l'information gain 

In [76]:
def info_gain(left, right, current_uncertainty):
    """Information Gain :
    The uncertainty of the starting node, minus the weighted impurity of
    two child nodes.
    """
    p = float(len(left)) / (len(left) + len(right))
    info_gain = current_uncertainty - p * gini(left) - (1 - p) * gini(right)
    return info_gain

In [78]:
# Calculez l'uncertainy de notre dataset.
current_uncertainty = gini(X)
current_uncertainty

# Vous devriez obtenir environ 0.8


0.7993999999999999

In [96]:
# Quel est notre information gain si nous posons la question 'Red' ?
true_rows, false_rows = partition(X, Question(0, "Red"))
info_gain(true_rows, false_rows, current_uncertainty)
# 0.1835

0.1834941572089112

In [91]:
# Quel est notre information gain si nous posons la question "Size >= 2"
true_rows, false_rows = partition(X, Question(1, 2))
info_gain(true_rows, false_rows, current_uncertainty)
# 0.1925

0.19248641975308622

### Pour ne pas tout tester manuellement, construisez la fonction vous permettant de trouver la question donnant le meilleur information gain en iterant sur chaque feature / value.

In [93]:
def find_best_split(rows):
    best_gain = 0
    best_question = None  
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)
            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [94]:
### Testez votre fonction
best_gain, best_question = find_best_split(X)
best_question
# Vous devriez avoir : is color == yellow ?

Is color == Yellow?

### Nous allons maintenant creer nos fonctions d'affichage

In [98]:
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [99]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [100]:
def build_tree(rows):
    """Builds the tree.

    Rules of recursion: 1) Believe that it works. 2) Start by checking
    for the base case (no further information gain). 3) Prepare for
    giant stack traces.
    """

    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [101]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [102]:
my_tree = build_tree(X)

In [103]:
print_tree(my_tree)

Is color == Yellow?
--> True:
  Predict {'Lemon': 20}
--> False:
  Is season == Winter?
  --> True:
    Is color == Green?
    --> True:
      Predict {'Kiwi': 22}
    --> False:
      Predict {'Orange': 20}
  --> False:
    Is season == Spring?
    --> True:
      Predict {'Apple': 19}
    --> False:
      Predict {'Grape': 19}


In [104]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [108]:
#######
# Demo:
# The tree predicts the 1st row of our
# training data is an apple with confidence 1.
classify(X[15], my_tree)
#######

{'Kiwi': 22}

In [109]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [110]:
#######
# Demo:
# Printing that a bit nicer
print_leaf(classify(X[0], my_tree))
#######

{'Grape': '100%'}

In [111]:
#######
# Demo:
# On the second example, the confidence is lower
print_leaf(classify(X[0], my_tree))
#######

{'Grape': '100%'}

In [112]:
# Evaluate
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [109]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Lemon': '100%'}
Actual: Grape. Predicted: {'Apple': '100%'}
Actual: Grape. Predicted: {'Apple': '100%'}
Actual: Lemon. Predicted: {'Lemon': '100%'}
